# CosmosDB change feed publishing to MediatR

This sample shows how to use the CosmosDB change feed with an Azure Function App (dotnet-isolated) worker to publish the events via MediatR to create an event-driven system.

In [ ]:
#!pwsh
dotnet pack ..\..\EventinatR.sln --output C:\Windows\Temp\EventinatR

Microsoft (R) Build Engine version 17.0.0-preview-21460-01+8f208e609 for .NET
Copyright (C) Microsoft Corporation. All rights reserved.

  Determining projects to restore...
  All projects are up-to-date for restore.


In [ ]:
#i "nuget:C:\Windows\Temp\EventinatR"
#r "nuget:EventinatR"
#r "nuget:EventinatR.Cosmos"
#r "nuget:MediatR"
#r "nuget:MediatR.Extensions.Microsoft.DependencyInjection"
#r "nuget:Microsoft.Azure.Cosmos"
#r "nuget:Microsoft.Azure.Functions.Worker"
#r "nuget:Microsoft.Azure.Functions.Worker.Sdk"
#r "nuget:Microsoft.Azure.Functions.Worker.Extensions.CosmosDB"
#r "nuget:Microsoft.Azure.Functions.Worker.Extensions.Http"
#r "nuget:System.Memory.Data"
#r "nuget:System.Linq.Async"

using System;
using System.Collections.Generic;
using System.Collections.ObjectModel;
using System.Linq;
using System.Net;
using System.Threading;
using EventinatR;
using EventinatR.CosmosDB;
using EventinatR.CosmosDB.Documents;
using EventinatR.Serialization;
using MediatR;
using Microsoft.Azure.Functions.Worker;
using Microsoft.Azure.Functions.Worker.Http;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;
using Microsoft.Extensions.Options;
using Newtonsoft.Json;

Restore sources C:\Windows\Temp\EventinatR Installed Packages EventinatR, 1.0.0 EventinatR.Cosmos, 1.0.0 MediatR, 9.0.0 MediatR.Extensions.Microsoft.DependencyInjection, 9.0.0 Microsoft.Azure.Cosmos, 3.21.0 Microsoft.Azure.Functions.Worker, 1.5.2 Microsoft.Azure.Functions.Worker.Extensions.CosmosDB, 3.0.9 Microsoft.Azure.Functions.Worker.Extensions.Http, 3.0.13 Microsoft.Azure.Functions.Worker.Sdk, 1.0.4 System.Linq.Async, 5.0.0 System.Memory.Data, 1.0.2

# Domain model

We are modeling a simple group with members, with the ability to add and remove members and the events that change the state of the group.

In [ ]:
public record GroupMember(string Name);
public record GroupId(string Name);

public abstract record Event : INotification;

public abstract record GroupEvent : Event
{
    public record Created(GroupId Id) : GroupEvent;
    public record AddedMember(GroupId Id, GroupMember Member) : GroupEvent;
    public record RemovedMember(GroupId Id, GroupMember Member) : GroupEvent;

    public static readonly EventDeserializer Deserializer = new(builder =>
    {
        builder.UseDefault<Created>();
        builder.UseDefault<AddedMember>();
        builder.UseDefault<RemovedMember>();
    });
}

public class Group
{
    private record GroupState(GroupId Id, IEnumerable<GroupMember> Members);

    public GroupId Id { get; private set; }
    public IEnumerable<GroupMember> Members => _members.AsEnumerable();

    private readonly List<GroupMember> _members = new();
    private readonly List<GroupEvent> _uncommittedEvents = new();

    private Group()
    {
    }

    private void AddEvent(GroupEvent e)
    {
        ApplyEvent(e);
        _uncommittedEvents.Add(e);
    }

    private void ApplyEvent(GroupEvent e)
    {
        switch (e)
        {
            case GroupEvent.Created created:
                Apply(created);
                break;
            case GroupEvent.AddedMember addedMember:
                Apply(addedMember);
                break;
            case GroupEvent.RemovedMember removedMember:
                Apply(removedMember);
                break;
            default:
                throw new InvalidOperationException($"Unsupported event: {e.GetType().FullName}");
        }
    }

    public static Group Create(string name)
    {
        var group = new Group();
        group.AddEvent(new GroupEvent.Created(new GroupId(name)));
        return group;
    }

    private void Apply(GroupEvent.Created e)
        => Id = e.Id;

    public void AddMember(string name)
    {
        if (!_members.Any(x => x.Name == name))
        {
            var member = new GroupMember(name);
            var e = new GroupEvent.AddedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.AddedMember e)
        => _members.Add(e.Member);

    public void RemoveMember(string name)
    {
        var member = _members.FirstOrDefault(x => x.Name == name);

        if (member is not null)
        {
            var e = new GroupEvent.RemovedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.RemovedMember e)
        => _members.Remove(e.Member);

    public static async Task<Group?> ReadAsync(EventStream stream)
    {
        var group = new Group();
        var snapshot = await stream.ReadSnapshotAsync<GroupState>();
        var state = snapshot.State;

        if (state is not null)
        {
            group.Id = state.Id;
            group._members.AddRange(state.Members);
        }

        var events = state is null
            ? await stream.ReadAsync().ToListAsync()
            : await snapshot.ReadAsync().ToListAsync();

        if (!events.Any() && state is null)
        {
            return null;
        }

        foreach (var e in events)
        {
            if (!GroupEvent.Deserializer.TryDeserialize<GroupEvent>(e, out var groupEvent))
            {
                throw new InvalidOperationException($"The event {e.Type} is not supported.");
            }

            group.ApplyEvent(groupEvent);
        }

        return group;
    }

    public async Task SaveAsync(EventStream stream)
    {
        if (_uncommittedEvents.Count > 0)
        {
            var version = await stream.AppendAsync(_uncommittedEvents);
            _uncommittedEvents.Clear();
            await stream.WriteSnapshotAsync(new GroupState(Id, _members), version);
        }
    }
}

# Projections

The CosmosDB change feed will be used to generate our projections from the event stream. MediatR is used to distribute the events to the appropriate handlers. This projection is an example, it is not recommended to use the snapshot capability of the event stream to store state (e.g. I was lazy for the example, be better than me, use EFCore or something else).

In [ ]:
public class GroupsProjection : INotificationHandler<GroupEvent.Created>
{
    private record State(List<GroupId> Groups);

    private readonly EventStore _store;

    public GroupsProjection(EventStore store)
        => _store = store ?? throw new ArgumentNullException(nameof(store));

    public static async IAsyncEnumerable<GroupId> ReadAsync(EventStore store)
    {
        var stream = await store.GetStreamAsync("groups");
        var snapshot = await stream.ReadSnapshotAsync<State>();
        var state = snapshot.State;

        if (state is not null)
        {
            foreach (var item in state.Groups)
            {
                yield return item;
            }
        }
    }

    public async Task Handle(GroupEvent.Created notification, CancellationToken cancellationToken)
    {
        var stream = await _store.GetStreamAsync("groups");
        var snapshot = await stream.ReadSnapshotAsync<State>();
        var state = snapshot.State ?? new State(new());

        state.Groups.Add(notification.Id);

        await stream.WriteSnapshotAsync(state, EventStreamVersion.None);
    }
}

# Dependency Injection

Use code like this to register the EventStore and MediatR.

In [ ]:
var host = new HostBuilder()
    .ConfigureFunctionsWorkerDefaults()
    .ConfigureServices(services =>
    {
        services.AddMediatR(typeof(Group).Assembly);

        services
            .AddOptions<CosmosEventStoreOptions>()
            .Configure<IConfiguration>((options, configuration) => configuration.GetSection(nameof(CosmosEventStore)).Bind(options));

        services.AddSingleton<EventStore>(serviceProvider =>
        {
            var options = serviceProvider.GetService<IOptions<CosmosEventStoreOptions>>().Value;
            return new CosmosEventStore(null, options);
        });

        services.AddSingleton(_ => new EventDeserializer(builder => builder.Use(GroupEvent.Deserializer)));
    })
    .Build();

# CosmosDBTrigger

Using the Function App CosmosDBTrigger binding, the changes to the event-store/events collection can be captured and replayed for asynchronous processing. We use the base Event class in the domain so that this one function can process all events generated by the domain.

In [ ]:
public class ChangeFeedEvents
{
    private readonly IMediator _mediator;
    private readonly EventDeserializer _deserializer;

    public ChangeFeedEvents(IMediator mediator, EventDeserializer deserializer)
    {
        _mediator = mediator ?? throw new ArgumentNullException(nameof(mediator));
        _deserializer = deserializer ?? throw new ArgumentNullException(nameof(deserializer));
    }

    [Function(nameof(ChangeFeedEvents))]
    public async Task RunAsync([CosmosDBTrigger(
        databaseName: CosmosEventStoreOptions.DefaultDatabaseId,
        collectionName: CosmosEventStoreOptions.DefaultContainerId,
        ConnectionStringSetting = "CosmosEventStore",
        LeaseCollectionName = "leases",
        CreateLeaseCollectionIfNotExists = true)] string json, FunctionContext context)
    {
        var documents = JsonConvert.DeserializeObject<ChangeFeedDocument[]>(json);
        
        foreach (var doc in documents.Where(x => x.IsEvent))
        {
            var e = doc.ToEventDocument().AsEvent();

            if (_deserializer.TryDeserialize<Event>(e, out var result))
            {
                await _mediator.Publish(result);
            }
        }
    }
}

# Groups API

Simple rest API for groups.

In [ ]:
public class Api
{
    private readonly EventStore _store;

    public Api(EventStore store)
        => _store = store ?? throw new System.ArgumentNullException(nameof(store));

    [Function(nameof(GetGroups))]
    public async Task<HttpResponseData> GetGroups(
        [HttpTrigger(AuthorizationLevel.Function, "get", Route = "groups")] HttpRequestData req)
    {
        var groups = await GroupsProjection.ReadAsync(_store).Select(x => x.Name).ToArrayAsync();

        var res = req.CreateResponse(HttpStatusCode.OK);
        
        await res.WriteAsJsonAsync(groups);

        return res;
    }

    [Function(nameof(GetGroup))]
    public async Task<HttpResponseData> GetGroup(
        [HttpTrigger(AuthorizationLevel.Function, "get", Route = "groups/{id}")] HttpRequestData req,
        string id)
    {
        var stream = await _store.GetStreamAsync(id);
        var group = await Group.ReadAsync(stream);

        if (group is null)
        {
            return req.CreateResponse(HttpStatusCode.NotFound);
        }

        var res = req.CreateResponse(HttpStatusCode.OK);

        await res.WriteAsJsonAsync(group);

        return res;
    }

    [Function(nameof(CreateGroup))]
    public async Task<HttpResponseData> CreateGroup(
        [HttpTrigger(AuthorizationLevel.Function, "post", Route = "groups/{id}")] HttpRequestData req,
        string id)
    {
        var stream = await _store.GetStreamAsync(id);
        var group = await Group.ReadAsync(stream);

        if (group is not null)
        {
            return req.CreateResponse(HttpStatusCode.Conflict);
        }

        group = Group.Create(id);

        await group.SaveAsync(stream);

        return req.CreateResponse(HttpStatusCode.Created);
    }

    [Function(nameof(GetGroupMembers))]
    public async Task<HttpResponseData> GetGroupMembers(
        [HttpTrigger(AuthorizationLevel.Function, "get", Route = "groups/{id}/members")] HttpRequestData req,
        string id)
    {
        var stream = await _store.GetStreamAsync(id);
        var group = await Group.ReadAsync(stream);

        if (group is null)
        {
            return req.CreateResponse(HttpStatusCode.NotFound);
        }

        var res = req.CreateResponse(HttpStatusCode.OK);

        await res.WriteAsJsonAsync(group.Members);

        return res;
    }

    [Function(nameof(AddGroupMember))]
    public async Task<HttpResponseData> AddGroupMember(
        [HttpTrigger(AuthorizationLevel.Function, "post", Route = "groups/{id}/members/{member}")] HttpRequestData req,
        string id,
        string member)
    {
        var stream = await _store.GetStreamAsync(id);
        var group = await Group.ReadAsync(stream);

        if (group is null)
        {
            return req.CreateResponse(HttpStatusCode.NotFound);
        }

        group.AddMember(member);
        await group.SaveAsync(stream);

        return req.CreateResponse(HttpStatusCode.OK);
    }

    [Function(nameof(RemoveGroupMember))]
    public async Task<HttpResponseData> RemoveGroupMember(
        [HttpTrigger(AuthorizationLevel.Function, "delete", Route = "groups/{id}/members/{member}")] HttpRequestData req,
        string id,
        string member)
    {
        var stream = await _store.GetStreamAsync(id);
        var group = await Group.ReadAsync(stream);

        if (group is null)
        {
            return req.CreateResponse(HttpStatusCode.NotFound);
        }

        group.RemoveMember(member);
        await group.SaveAsync(stream);

        return req.CreateResponse(HttpStatusCode.OK);
    }
}